<a href="https://colab.research.google.com/github/GM-Praveen/Projects/blob/main/Smart_Document_Q%26A_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio langchain transformers sentence-transformers faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 42.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# Global state
db = None
qa_chain = None
chat_history = []

# Process PDF file
def process_pdf(pdf_file):
    global db, qa_chain

    try:
        file_path = pdf_file.name
        loader = PyPDFLoader(file_path)
        documents = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = text_splitter.split_documents(documents)

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        db = FAISS.from_documents(docs, embeddings)

        hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-large", max_length=512)
        llm = HuggingFacePipeline(pipeline=hf_pipeline)

        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            return_source_documents=True
        )

        return "✅ PDF uploaded and processed successfully!"

    except Exception as e:
        return f"❌ Error processing PDF: {e}"

In [ ]:
# Ask question
def ask_question(query, mode, tone):
    global chat_history, qa_chain

    if qa_chain is None:
        return chat_history + [("System", "❗ Please upload and process a PDF first.")]

    if not query.strip():
        return chat_history + [("System", "❗ Please enter a valid question.")]

    if mode == "Semantic Search":
        docs = db.similarity_search(query, k=3)
        context = "\n\n---\n\n".join([doc.page_content for doc in docs])
        response = f"🔍 Top relevant sections:\n\n{context}"
    else:
        tone_prompt = {
            "Simple": "Explain in simple terms:",
            "Technical": "Explain in a technical and detailed way:",
            "Bullet Points": "Summarize using bullet points:"
        }.get(tone, "Explain clearly:")

        prompt = f"{tone_prompt}\n\n{query}"
        try:
            result = qa_chain({"query": prompt})
            response = result["result"]
        except Exception as e:
            response = f"⚠️ Error: {e}"

    chat_history.append((query, response))
    return chat_history

In [ ]:
dark_theme = gr.themes.Base(
    primary_hue="teal", secondary_hue="indigo", neutral_hue="gray"
).set(
    button_primary_background_fill="#0d9488",
    button_primary_background_fill_hover="#0f766e",
    button_secondary_background_fill="#334155",
    button_secondary_text_color="#f1f5f9",
    block_border_color="#475569"
)

custom_css = """
.gradio-container {
    font-family: 'Segoe UI', sans-serif;
    background-color: #1e293b;
    color: #e2e8f0;
}

#header-text {
    text-align: center;
    font-size: 32px;
    font-weight: bold;
    color: #f1f5f9;
    font-family: 'Georgia', serif;
}

#subheader-text {
    text-align: center;
    font-size: 17px;
    color: #cbd5e1;
    margin-bottom: 25px;
    font-style: italic;
}

.group-box {
    background-color: #334155;
    border-radius: 12px;
    padding: 22px;
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
    margin-bottom: 20px;
}

textarea, input[type="text"] {
    background-color: #1e293b !important;
    border: 1px solid #475569 !important;
    border-radius: 8px !important;
    padding: 10px !important;
    font-size: 15px !important;
    color: #e2e8f0 !important;
}
"""

# Gradio App
with gr.Blocks(theme=dark_theme, css=custom_css) as demo:
    gr.Markdown("<div id='header-text'>📄 Smart PDF Q&A Assistant</div>")
    gr.Markdown("<div id='subheader-text'>Upload a PDF, ask questions, and get contextual answers from the document.</div>")

    with gr.Row():
        with gr.Column(scale=1):
            with gr.Group(elem_classes="group-box"):
                file_input = gr.File(label="📁 Upload PDF", file_types=[".pdf"])
                process_btn = gr.Button("🚀 Process PDF", variant="primary")
                status_output = gr.Textbox(label="📌 Status", interactive=False)
                query = gr.Textbox(label="🔎 Ask a Question")
                mode = gr.Radio(["QA Mode", "Semantic Search"], label="Mode", value="QA Mode")
                tone = gr.Dropdown(["Simple", "Technical", "Bullet Points"], label="Answer Style", value="Bullet Points")
                ask_btn = gr.Button("🧠 Get Answer")

        with gr.Column(scale=1):
            with gr.Group(elem_classes="group-box"):
                chatbot = gr.Chatbot(label="💬 Q&A Chat History")

    process_btn.click(process_pdf, inputs=file_input, outputs=status_output)
    ask_btn.click(ask_question, inputs=[query, mode, tone], outputs=chatbot)

demo.launch()

/tmp/ipykernel_36/884365836.py:70: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="💬 Q&A Chat History")


* Running on local URL:  http://127.0.0.1:7864
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://6fc5de24471272d542.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
